In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time
import sys, os

In [2]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    elif t=='each':
        x=x.dropna().values
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [3]:
# RSI, %time above RSI
def RSI(series, period=14):
 #print(len(series))
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])
 #rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / pd.stats.moments.ewma(d, com=period-1, adjust=False)
 rs = pd.Series.ewm(u, com=period-1).mean()/pd.Series.ewm(d, com=period-1).mean()
 rsi = 100 - 100 / (1 + rs)
 return np.round(rsi.values[-1],2),get_percentile(rsi)#np.round(100*np.sum(np.where(rsi>rsi.values[-1],1,0))/len(rsi),2)

In [4]:
def consecutive(x,lag=4):
    c=0
    for i in x[-lag:][::-1]:
        if i>=0:
            c+=1
        else:
            return c
    return c    

def consecutive_increases(x):
    c=0
    latest=x.values[-1]
    for i in x[::-1]:
        if i==latest:
            continue
        if latest>i:
            c+=1
            latest=i
        else:
            return c
    return c    

In [5]:
# Consecutive increases, hist percentile, hist change mean 4 week, slope of 4 week macd
def macd_calcs(x):
    macd=x.ewm(span=12,min_periods=0,adjust=True,ignore_na=True).mean()-x.ewm(span=26,min_periods=0,adjust=True,ignore_na=True).mean()
    signal=macd.ewm(span=9,min_periods=0,adjust=True,ignore_na=True).mean()
    macd_hist=macd-signal
    return consecutive_increases(macd_hist), get_percentile(macd_hist),(macd_hist/macd_hist.shift(1)-1).rolling(4).mean().values[-1:][0],np.polyfit(x=[1,2,3,4],y=macd.values[-4:],deg=1,full=False)[0]

In [6]:
def get_intraday(ticker):
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': False}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['close'] is None)==False):
            return ([i['open'],i['high'],i['low'],i['close'],i['close']])
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': True}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['marketClose'] is None)==False):
            return([i['marketOpen'],i['marketHigh'],i['marketLow'],i['marketClose'],i['marketClose']])

In [197]:
holdings='GDXJ,AMZN,AAPL,FSLY,PTON,FUTU,BLCN,NOMD,GDXJ,BGS,AWK,SHW,EAT,CF,BATT,CAKE,TTEK,NWL,FAST,FXE,URA,ITB,GOLD,SPY,DOCU,ZM,PTON,MSFT,NVDA,EA,CRM,VEEV,ADBE,QQQ,TSLA,W,ESTC,CLX,GDX,CDNS,MELI,ARKF,TTD,WK,SNAP,ESPO,NEM,YETI,FCX'


In [198]:
holdings=holdings.split(',')

In [199]:
#holdings=['GDX','RHHBY','ETSY','ECL','OLLI','BLK','SPY','SPXL','AZN','CDNS','CWH','BYND']
#holdings=['TSLA']

In [200]:
res=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

In [201]:
holdings

['GDXJ',
 'AMZN',
 'AAPL',
 'FSLY',
 'PTON',
 'FUTU',
 'BLCN',
 'NOMD',
 'GDXJ',
 'BGS',
 'AWK',
 'SHW',
 'EAT',
 'CF',
 'BATT',
 'CAKE',
 'TTEK',
 'NWL',
 'FAST',
 'FXE',
 'URA',
 'ITB',
 'GOLD',
 'SPY',
 'DOCU',
 'ZM',
 'PTON',
 'MSFT',
 'NVDA',
 'EA',
 'CRM',
 'VEEV',
 'ADBE',
 'QQQ',
 'TSLA',
 'W',
 'ESTC',
 'CLX',
 'GDX',
 'CDNS',
 'MELI',
 'ARKF',
 'TTD',
 'WK',
 'SNAP',
 'ESPO',
 'NEM',
 'YETI',
 'FCX']

In [202]:
c=1
for ticker in holdings:
    print(ticker)
    time.sleep(1)
    c+=1
    if c==4:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey=YEUC2DL0F7KC43PB&outputsize=full'.format(ticker)
    response = requests.get(
        url
#        params=d,
        )
    #print(response.text)
    #print(url)
    try:
        spy=[[],[],[],[],[],[],[]]
        #print(response.text)
        for key, val in response.json()['Time Series (Daily)'].items():
            spy[0].append(float(val['1. open']))
            spy[1].append(float(val['2. high']))
            spy[2].append(float(val['3. low']))
            spy[3].append(float(val['4. close']))
            spy[4].append(float(val['5. adjusted close']))
            #spy[5].append(float(val['6. volume']))
            #spy[6].append(float(val['7. dividend amount']))
        t=list(response.json()['Time Series (Daily)'].keys())
        
        try:
            if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
                x=get_intraday(ticker)
                spy[0].insert(0,float(x[0]))
                spy[1].insert(0,float(x[1]))
                spy[2].insert(0,float(x[2]))
                spy[3].insert(0,float(x[3]))
                spy[4].insert(0,float(x[4]))
                t.insert(0,str(pd.Timestamp.today().date()))

                #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
        except:
            print(ticker,'IEX')

        


        spy=pd.DataFrame({'Date':t,
                     'open':spy[0],
                      'high':spy[1],
                      'low':spy[2],
                      'close':spy[3],
                      'adj_close':spy[4]#,
                    #  'volume':spy[5],
                    #  'dividend':spy[6],
                     })
        spy=spy.astype({'Date': 'datetime64'})
        spy=spy.sort_values(by='Date')
        spy['highest_high']=spy.high.rolling(22).max()
        spy['lowest_low']=spy.low.rolling(22).min()
        spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
        spy['k']=spy.k.rolling(3).mean()
        spy['k_change'] = spy.k/spy.k.shift(3)-1
        res[0].append(ticker)
        res[1].append(spy.iloc[-1]['Date']) 
        res[2].append(spy.iloc[-1]['k']) 
        res[3].append(spy.iloc[-1]['k_change'])
        res[4].append(np.nanpercentile(spy.k_change,q=10))
        res[5].append(get_percentile(spy.k_change))
        temp_res=RSI(spy.adj_close)
        res[6].append(temp_res[0])
        res[7].append(temp_res[1])
        temp_res=macd_calcs(spy.adj_close)
        res[8].append(temp_res[0])
        res[9].append(temp_res[1])
        res[10].append(spy.adj_close.rolling(90).std().values[-1])
        res[11].append((spy.adj_close/spy.adj_close.shift(5)-1).rolling(8).mean().values[-1])
        res[12].append((spy.adj_close/spy.adj_close.shift(22)-1).values[-1])
        res[13].append((spy.adj_close/spy.adj_close.shift(5)-1).values[-1])
        res[14].append((spy.adj_close/spy.adj_close.shift(1)-1).values[-1])
        time.sleep(1)
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(ticker,exc_type, exc_tb.tb_lineno)  


GDXJ
AMZN
AAPL
FSLY
PTON
FUTU
BLCN
BLCN IEX
NOMD
GDXJ
BGS
AWK
SHW
EAT
CF
BATT
BATT IEX
CAKE
TTEK
NWL
FAST
FXE
FXE IEX
URA
URA IEX
ITB
GOLD
SPY
DOCU
ZM
PTON
MSFT
NVDA
EA
CRM
VEEV
ADBE
QQQ
TSLA
W
ESTC
CLX
GDX
CDNS
MELI
ARKF
ARKF IEX
TTD
WK
WK IEX
SNAP
ESPO
NEM
YETI
FCX


In [203]:
res=pd.DataFrame({'ticker':res[0], 
            'date':res[1],
             'k':res[2],
             'k_change':res[3],
             'k_10_percentile':res[4],
             'k_change_percentile':res[5],
             'RSI':res[6],
             'RSI_percentile':res[7],
             'hist_cons':res[8],    
             'hist_percentile':res[9],
             '90dayStd':res[10],
             'ret1wk_mean':res[11],
             '22dayret':res[12],
             '5dayret':res[13],
             '1dayret':res[14]         
             }
            )

In [204]:
res.sort_values(by='k_change',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret,5dayret,1dayret
34,TSLA,2020-08-06,0.397916,-0.250588,-0.449328,19.85,59.05,67.17,4,0.75,330.439094,-0.018115,0.088584,0.017136,0.018828
36,ESTC,2020-08-06,0.642266,-0.173675,-0.482762,25.46,50.53,46.65,0,45.24,14.987095,0.031595,-0.010930,-0.024519,-0.025652
39,CDNS,2020-08-06,0.853435,-0.128902,-0.446172,30.85,71.04,93.19,0,96.29,11.714744,0.038844,0.120381,0.027961,0.009211
3,FSLY,2020-08-06,0.724624,-0.101229,-0.493447,33.57,52.38,48.14,0,88.03,28.249613,0.182958,0.033939,-0.022472,-0.169299
24,DOCU,2020-08-06,0.872990,-0.095395,-0.369440,28.47,62.49,65.54,0,82.40,42.588627,0.077797,0.097193,0.014163,-0.040041
17,NWL,2020-08-06,0.574317,-0.094477,-0.473968,35.74,55.87,63.98,1,18.18,1.896037,-0.013480,0.064669,-0.047686,0.006862
7,NOMD,2020-08-06,0.906250,-0.055136,-0.389021,39.17,81.66,99.21,1,97.84,1.107150,0.052704,0.146226,0.066725,0.041131
19,FXE,2020-08-05,0.900689,-0.047818,-0.539193,42.12,75.10,97.82,0,78.30,2.733977,0.013434,0.047793,0.007025,0.004943
47,YETI,2020-08-06,0.925293,-0.047304,-0.455267,42.18,70.52,87.79,0,73.21,9.346580,0.057819,0.207500,0.055903,-0.017722
18,FAST,2020-08-06,0.945144,-0.017492,-0.471513,47.28,73.24,95.95,0,94.70,4.581761,0.026735,0.091892,0.020038,-0.004266


In [211]:
res.sort_values(by='ret1wk_mean',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret,5dayret,1dayret
44,SNAP,2020-08-06,0.163686,0.122110,-0.500354,66.27,41.55,29.14,3,3.58,4.069772,-0.034151,-0.137848,-0.045515,-0.014286
20,URA,2020-08-05,0.552381,-0.005047,-0.574981,51.42,57.00,76.92,2,31.68,0.859510,-0.022081,0.065041,0.002551,0.005115
34,TSLA,2020-08-06,0.397916,-0.250588,-0.449328,19.85,59.05,67.17,4,0.75,330.439094,-0.018115,0.088584,0.017136,0.018828
17,NWL,2020-08-06,0.574317,-0.094477,-0.473968,35.74,55.87,63.98,1,18.18,1.896037,-0.013480,0.064669,-0.047686,0.006862
15,CAKE,2020-08-06,0.578473,0.021776,-0.471773,52.68,56.45,65.88,1,61.28,2.871522,0.005785,0.184747,-0.016903,0.035611
48,FCX,2020-08-06,0.766555,0.156109,-0.495960,67.34,72.57,96.39,2,44.36,2.180906,0.006941,0.157502,0.138717,0.054760
14,BATT,2020-08-05,0.860130,0.065130,-0.707096,55.38,69.29,94.92,2,47.25,1.130886,0.010916,0.103378,0.036538,0.034648
23,SPY,2020-08-06,0.994777,0.074507,-0.452089,63.51,68.96,91.06,5,90.78,20.670424,0.012644,0.064663,0.031208,0.005902
19,FXE,2020-08-05,0.900689,-0.047818,-0.539193,42.12,75.10,97.82,0,78.30,2.733977,0.013434,0.047793,0.007025,0.004943
10,AWK,2020-08-06,0.932489,0.000047,-0.410368,49.43,70.27,92.86,0,29.43,9.055953,0.014939,0.126391,0.001023,-0.003192


In [210]:
res[(res.RSI_percentile<93) & (res.ret1wk_mean>=0.023557) & (res['1dayret']>=-0.02)].sort_values(by='ret1wk_mean',ascending=False)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret,5dayret,1dayret
4,PTON,2020-08-06,0.839786,0.000315,-0.360403,47.92,64.98,75.49,0,73.39,13.290430,0.075313,0.170832,0.090839,0.000000
26,PTON,2020-08-06,0.838340,-0.001408,-0.360403,47.40,64.82,74.51,0,73.39,13.289148,0.075188,0.169763,0.089844,-0.000913
47,YETI,2020-08-06,0.925293,-0.047304,-0.455267,42.18,70.52,87.79,0,73.21,9.346580,0.057819,0.207500,0.055903,-0.017722
28,NVDA,2020-08-06,0.973639,0.083316,-0.467076,61.70,71.57,92.27,7,98.74,55.454733,0.047776,0.147770,0.067505,0.003876
12,EAT,2020-08-06,0.889960,0.188841,-0.481074,69.38,62.30,81.67,2,95.92,4.978542,0.044607,0.292287,0.025779,0.022484
30,CRM,2020-08-06,0.918590,0.366831,-0.445331,80.64,68.97,90.06,9,98.77,17.275553,0.040487,0.068566,0.094653,0.035556
13,CF,2020-08-06,0.961588,0.090712,-0.450388,63.25,67.11,88.11,3,92.66,2.062149,0.038465,0.152977,0.093476,0.023390
31,VEEV,2020-08-06,0.953326,0.017907,-0.393010,54.27,68.08,87.71,0,89.26,33.388122,0.033722,0.105450,0.033748,-0.005107
46,NEM,2020-08-06,0.897594,0.171454,-0.488544,66.88,66.40,91.54,0,90.83,5.312570,0.032088,0.124620,0.069982,-0.001634
25,ZM,2020-08-06,0.738128,0.608191,-0.392089,88.12,59.82,61.27,9,56.53,51.408029,0.029419,0.023846,0.073400,-0.016673


In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1
    https://cloud.iexapis.com/stable/stock/CWH/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=30&chartIEXWhenNull=True

In [ ]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':'MBUU','interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

In [ ]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response2 = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format('MBUU')#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [ ]:
t=pd.DataFrame(response.json()[0]['date'],response.json()[0]['open'],response.json()[0]['high'],response.json()[0]['low'],response.json()[0]['close'],response.json()[0]['close'])

In [ ]:
date=list(response.json()['Time Series (Daily)'].keys())

In [ ]:
date.insert(0,response2.json()[0]['date'])

In [ ]:
date

In [ ]:
spy


In [ ]:
np.nanpercentile(spy.k_change,q=10)

In [ ]:
response.json()[0]

In [ ]:
pd.to_datetime('today')

In [ ]:
pd.Timestamp.today().date()

In [ ]:
pd.to_datetime(t[0]).date()

In [ ]:
pd.to_datetime(t[0]).date()==pd.Timestamp.today().date()

In [ ]:
spy.adj_close.rolling(90).std().values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(22)-1).values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1]

In [95]:
spy=spy.set_index('Date')#['2020-07-01':]

In [96]:
spy['ret']=spy.adj_close/spy.adj_close.shift(5)-1

In [97]:
#spy['ret_3']=spy.ret.rolling(3).mean()

In [98]:
np.nanpercentile(spy.k_change,q=10),np.nanpercentile(spy.ret,q=10)

(-0.44933637984606634, -0.07577710062814896)

In [99]:
def each_percentile(x,val):
    return (x<val).sum()/len(x)

In [100]:
spy['ret_p']=spy.ret.apply(lambda x: each_percentile(spy.ret,x))

In [101]:
#spy['ret_3_p']=spy.ret_3.apply(lambda x: each_percentile(spy.ret,x))

In [102]:
spy['2020-07-01':]

,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change,ret,ret_p
Date,,,,,,,,,,,
2020-07-01,1083.00,1135.3300,1080.5000,1119.63,1119.63,1135.33,858.4400,0.942787,0.254790,0.165250,0.967374
2020-07-02,1221.48,1228.0000,1185.6000,1208.66,1208.66,1228.00,858.4400,0.952411,0.167343,0.225846,0.982704
2020-07-06,1276.69,1377.7900,1266.0400,1371.58,1371.58,1377.79,858.4400,0.959670,0.113715,0.429116,0.995676
2020-07-07,1405.01,1429.5000,1336.7100,1389.86,1389.86,1429.50,866.2000,0.955113,0.013074,0.376985,0.994497
2020-07-08,1405.00,1417.2600,1311.3400,1365.88,1365.88,1429.50,908.5000,0.931853,-0.021584,0.264926,0.987421
2020-07-09,1396.99,1408.5600,1351.2800,1394.28,1394.28,1429.50,908.5000,0.913306,-0.048313,0.245304,0.985849
2020-07-10,1396.00,1548.9200,1376.0100,1544.65,1544.65,1548.92,908.5000,0.934540,-0.021540,0.277986,0.989387
2020-07-13,1659.00,1794.9900,1471.1100,1497.06,1497.06,1794.99,908.5000,0.863218,-0.073655,0.091486,0.889151
2020-07-14,1556.00,1590.0000,1431.0000,1516.80,1516.80,1794.99,908.5000,0.781148,-0.144703,0.091333,0.888758


In [ ]:
spy['2020-02-20':'2020-3-31']


So something can go down at random 
look to see how other stocks have done. if it is aleader like CWH or MELI NEED TO track and buy
that is part of the in-cycle market leader

In [148]:
import requests
from bs4 import BeautifulSoup
import json
import time

In [32]:
fetch_these=['Earnings','Market Cap','P/E','Forward P/E','Sales','Income','Dividend %','Short Float', 'Target Price'
             ,'ROE','Perf Week','Perf Month','Perf Quarter','Perf Year','Insider Own','Insider Trans']

In [33]:

def fetcher(soup, fetch_these,ticker):
    temp=[]
    temp.append(ticker)
    temp.append(pd.datetime.now())
    for i in fetch_these:
        x=soup.find('table', class_='snapshot-table2').find('td', string=i).find_next('td').text
        temp.append(x)
    return temp

In [34]:
stock_stats=[]
for j in holdings:
    try:
        url=str.format('https://finviz.com/quote.ashx?t={}',j)
        r=requests.get(url)
        soup=BeautifulSoup(r.text)
        stock_stats.append(fetcher(soup,fetch_these,j))
        time.sleep(np.random.randint(1,5))
        if np.random.uniform()<=0.3:
            time.sleep(4)
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(j, exc_type, exc_tb.tb_lineno)

W <class 'AttributeError'> 7


/Users/rittikghosh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [35]:
stock_stats

[]

In [39]:
j='W'
url=str.format('https://finance.yahoo.com/quote/W?p=W&.tsrc=fin-srch')
r=requests.get(url)

In [41]:
soup=BeautifulSoup(r.text)

In [ ]:
https://cloud.iexapis.com/stable/stock/W/upcoming-earnings?token=pk_9469e57bc642462d905e4dc0003523ab
    http://q4finance.com/analysis/earning%20Drift/BA

In [42]:
url='http://q4finance.com/analysis/earning%20Drift/BA'
r=requests.get(url)

In [44]:
soup=BeautifulSoup(r.text)

In [46]:
soup.find('table', class_='table table-responsive img-fluid  table_text_nowrap').text

AttributeError: 'NoneType' object has no attribute 'text'

In [109]:
url='https://service.q4finance.com/analysis/earning_drift_data'
r=requests.post(url, data={'identifier':'AAPL'})

In [110]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

2019-07-30
2019-04-30
2019-01-29
2018-11-01
2018-07-31
2018-05-01


In [94]:
for d in json.loads(r.text)['data']['data']:
    print(d['date'])

2019-07-30
2019-04-30
2019-01-29
2018-11-01
2018-07-31
2018-05-01


In [261]:
df=pd.DataFrame()
tickers=pd.read_csv('stock_names.csv')['ticker']
#tickers=['MSFT','AAPL','BLK','HD']

In [262]:
c=0
for ticker in tickers:
    c+=1
    url='https://service.q4finance.com/analysis/historic_return_graph'
    r=requests.post(url, data={'identifier':ticker})
    peer_of=[]
    peers=[]
    for s in json.loads(r.text)['response_data']:
        if s['identifier'] == ticker:
            continue
        peers.append(s['identifier'])
        peer_of.append(ticker)
    df=df.append(pd.DataFrame(zip(peer_of,peers)),ignore_index=True)
    if c==3:
        time.sleep(2)
        c=0

In [264]:
df.iloc[:,0].unique

0           A
1           A
2           A
3           A
4           A
         ... 
15693      LE
15694      LE
15695    XONE
15696    XONE
15697    XONE
Name: 0, Length: 15698, dtype: object

In [167]:
tickers=pd.read_csv('stock_names.csv')['ticker']

In [168]:
fields=['identifier','ceo','company_url','industry_category','industry_group','sec_filings']

In [169]:
store=[[],[],[],[],[],[]]
c=0
for t in tickers:
    c+=1
    url='https://service.q4finance.com/analysis/primary_info_latest_data'
    r=requests.post(url, data={'identifier':t})
    for idx,f in enumerate(fields):
        store[idx].append(json.loads(r.text)['static_data'][f])
    if c==15:
        time.sleep(2)
        c=0

In [170]:
data=pd.DataFrame({
    'ticker':store[0]
    ,'ceo':store[1]
    ,'website':store[2]
    ,'industry_category':store[3]
    ,'industry_group':store[4]
    ,'sec_filings':store[5]
})

In [171]:
data.to_csv('stock_data.csv')

In [172]:
data

,ticker,ceo,website,industry_category,industry_group,sec_filings
0,A,Michael R. McMullen,www.agilent.com,Health Services,Medical Laboratories & Research,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
1,AA,Roy C. Harvey,www.alcoa.com,Metals & Mining,Aluminum,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
2,AABA,Marissa Ann Mayer,www.yahoo.com,Internet,Internet Information Providers,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
3,AAL,Doug Parker,www.aa.com,Transportation,Major Airlines,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
4,AAN,John W. Robinson,www.aarons.com,Diversified Services,Rental & Leasing Services,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
...,...,...,...,...,...,...
2377,EB,N/A,N/A,nm,nm,https://www.sec.gov/cgi-bin/browse-edgar?CIK=E...
2378,AMSC,Daniel Patrick McGahn,www.amsc.com,Manufacturing,Industrial Electrical Equipment,https://www.sec.gov/cgi-bin/browse-edgar?CIK=A...
2379,RRD,Daniel L. Knotts,www.rrdonnelley.com,Diversified Services,Business Services,https://www.sec.gov/cgi-bin/browse-edgar?CIK=R...
2380,LE,Jerome Squire Griffith,www.landsend.com,Retail,Catalog & Mail Order Houses,https://www.sec.gov/cgi-bin/browse-edgar?CIK=L...


In [140]:
for f in fields:
    print(json.loads(r.text)['static_data'][f])

AAPL
Tim Cook
www.apple.com
Consumer Durables
Electronic Equipment
https://www.sec.gov/cgi-bin/browse-edgar?CIK=AAPL&owner=exclude&action=getcompany


In [130]:
store=[[],[]]
for etf in json.loads(r.text)['data']:
    store[0].append(etf['identifier'])
    store[1].append(etf['name'])

In [132]:
pd.DataFrame(zip(store[0],store[1]),columns=['ticker','name']).to_csv('stock_names.csv')